In [5]:
import numpy as np
import os
import pandas as pd
from sklearn.cluster import KMeans
from skimage import io
from moviepy.editor import VideoFileClip
import cv2

In [ ]:
def get_frame(image):
    r, c, d = image.shape
    image = image.reshape(r * c, d)
    kmeans = KMeans(n_clusters=32, n_init=1, max_iter=200)
    kmeans.fit(image)

    clusters = np.asarray(kmeans.cluster_centers_, dtype=np.uint8)
    labels = np.asarray(kmeans.labels_, dtype=np.uint8)
    labels = labels.reshape(r, c)
    
    return clusters[labels]

In [6]:
df = pd.read_csv("/kaggle/input/robotcsv/output_1 (1).csv")
df.head()
arr1 = df["SHOT_INDEX"].tolist()
print(len(arr1))

458


In [ ]:
compressed_dir_path = "/kaggle/working/compressed_data"
frame_dir = "/kaggle/input/ml-project-frames/RobotFrames"
os.makedirs(compressed_dir_path, exist_ok=True)

# k = 0
for i in arr1:
#     if k == 1:
#         break
    frame_path = os.path.join(frame_dir, f"frame_{i:04d}.jpg")
    image = io.imread(frame_path)
    io.imsave(os.path.join(compressed_dir_path, f"{i}.jpg"), get_frame(image))
#     k += 1

In [8]:
import shutil

# Define the directory to zip
directory_to_zip = '/kaggle/working/compressed_data'

# Define the output zip file path
zip_file_path = '/kaggle/working/compressed_data.zip'

# Create a zip file of the directory
shutil.make_archive(zip_file_path.split('.zip')[0], 'zip', directory_to_zip)

'/kaggle/working/compressed_data.zip'

In [22]:
# clusters_path = "D:/iiitd/sem_1/ML/Project/bw_data/clusters"
# labels_path = "D:/iiitd/sem_1/ML/Project/bw_data/labels"

# x = os.listdir(clusters_path)
# y = os.listdir(labels_path)
# x.sort()
# y.sort()
# print(x)
# print(y)

output_path = "/kaggle/working/decompressed_video.mp4"

# img = cv2.imread(os.path.join(labels_path, y[0]))
# height, width = img.shape
r = 800
c = 1920

fps = 24
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(output_path, fourcc, fps, (c, r))

arr = arr1.copy()
# print(arr)
arr.append(14345)
for i in range(len(arr) - 1):
    frame_path = os.path.join("/kaggle/input/compressed-frame", f"{arr[i]}.jpg")
    img = cv2.imread(frame_path)
#     img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
#     io.imshow(img)
#     print(img.shape)
    for j in range(arr[i], arr[i + 1]):
        video.write(img)
# (1608, 3840, 3)
video.release()

In [4]:
pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for imageio_ffmpeg>=0.2.0 from https://files.pythonhosted.org/packages/1a/98/3df1d8dd8f2c121b6c588b1e0d604f36592d56df9c41fb155ed546c6a5ed/imageio_ffmpeg-0.4.9-py3-none-manylinux2010_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 45.1 MB/s eta 0:00:00:00:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110721 sha256=bd952ada392079d7a467052e850be13b6e8384cccde927c91dc985da2cb1fb81
  Stored in directory: /root/.cache/pip/wheels/96/32/2d/e10123bd88fbfc02fed53cc18c80a171d3c87479ed845fa7c1
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are inst

In [23]:
from moviepy.editor import VideoFileClip

def extract_audio(input_video, output_audio):
    video_clip = VideoFileClip(input_video)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(output_audio)

# Example usage
input_video_file = "/kaggle/input/ml-projectdatet/VideoCompressionDataset/AlitaBattleAngel.mkv"
output_audio_file = "robot_audio.mp3"
extract_audio(input_video_file, output_audio_file)

MoviePy - Writing audio in robot_audio.mp3


MoviePy - Done.


In [24]:
import numpy as np
from sklearn.decomposition import PCA
import librosa

# Load audio data
file_path = "/kaggle/working/robot_audio.mp3"
audio_data, sr = librosa.load(file_path, sr=None)

# Reshape the audio data into a 2D array (features x time steps)
audio_data_2d = audio_data.reshape(1, -1)

# Choose the number of principal components (compression size)
compression_size = 0.3  # Adjust based on the number of features
pca = PCA(n_components=compression_size)

# Fit the PCA model on the audio data
compressed_representation1 = pca.fit_transform(audio_data_2d)

# Save the compressed representation and metadata
np.save("compressed_audio.npy", compressed_representation1)

In [25]:
import soundfile as sf
# Decompress audio
compressed_audio = np.load("/kaggle/working/compressed_audio.npy")
reconstructed_audio_2d = pca.inverse_transform(compressed_audio)
reconstructed_audio = reconstructed_audio_2d.flatten()

# Save the reconstructed audio
sf.write("reconstructed_robot_audio.mp3", reconstructed_audio, sr)

In [29]:
def merge_audio_video(video_path, audio_path, output_path):
    # Load the video clip
    video_clip = VideoFileClip(video_path)

    # Load the audio clip
    audio_clip = AudioFileClip(audio_path)

    # Set the video clip's audio to the loaded audio clip
    video_clip = video_clip.set_audio(audio_clip)

    # Write the result to a new video file
    video_clip.write_videofile(output_path, codec="mpeg4", audio_codec="libmp3lame", temp_audiofile="temp_audio.mp3", remove_temp=True)

    
video_path = "/kaggle/working/decompressed_video.mp4"
audio_path = "/kaggle/working/reconstructed_robot_audio.mp3"
output_path = "decompressed_robot_video.mp4"
merge_audio_video(video_path, audio_path, output_path)

Moviepy - Building video decompressed_robot_video.mp4.
MoviePy - Writing audio in decompressed_robot_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video decompressed_robot_video.mp4



TypeError: must be real number, not NoneType